In [3]:
import uuid
import os
from datetime import datetime
import pandas as pd
import numpy as np
import openai
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Distance, VectorParams

In [4]:
load_dotenv()

True

In [12]:
client = QdrantClient("http://localhost:6333")
collection_name = "admission_records"

In [19]:
client.delete_collection(collection_name=collection_name)

True

In [20]:
def create_collection():
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=1536,  # OpenAI embedding size for text-embedding-3-small
            distance=Distance.COSINE  # Choose from COSINE, EUCLIDEAN, DOT
        )
    )
    print(f"Collection '{collection_name}' created successfully.")

In [21]:
create_collection()

Collection 'admission_records' created successfully.


In [6]:
def generate_openai_embedding(text):
    try:
        response = openai.embeddings.create(
            input=text,
            model="text-embedding-3-small"
        )
        embedding = response.data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return None


In [7]:
# Search data in Qdrant
def search_similar_vectors(query_text, top_k=5):
    query_embedding = generate_openai_embedding(query_text)
    if not query_embedding:
        print("Failed to generate query embedding.")
        return

    search_results = client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=top_k
    )

    for result in search_results:
        print(f"Found ID: {result.id}, Score: {result.score}, Metadata: {result.payload}")

In [18]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

query = "เกณฑ์ของภาควิศวกรรมซอฟต์แวร์และความรู้ รอบ1/1 นานาชาติและภาษาอังกฤษ"
search_result = client.search(
    collection_name=collection_name,
    query_vector=generate_openai_embedding(query),
    query_filter=Filter(
        must=[
            FieldCondition(key="admission_round", match=MatchValue(value="1/1")),
            FieldCondition(key="admission_program", match=MatchValue(value="นานาชาติและภาษาอังกฤษ")),
        ]
    ),
    with_payload=True,
    limit=5,
)

for result in search_result:
        print(f"Found ID: {result.id}, Score: {result.score}, Metadata: {result.payload}")

Found ID: 4ca8c30e-632e-51bf-932d-8792521ce5a8, Score: 0.34026676, Metadata: {'major': 'วศ.บ. สาขาวิชาวิศวกรรมซอฟต์แวร์และความรู้ (นานาชาติ)', 'admission_round': '1/1', 'admission_program': 'นานาชาติและภาษาอังกฤษ', 'reference': 'https://admission.ku.ac.th/majors/project/3/', 'created_at': '2025-01-25T16:37:01.837049', 'contents': 'N/A'}
Found ID: dd405448-8a3c-55bf-9244-031e4b5f69cf, Score: 0.33196616, Metadata: {'major': 'วศ.บ. สาขาวิชาวิศวกรรมสิ่งแวดล้อม (หลักสูตรภาษาอังกฤษ)', 'admission_round': '1/1', 'admission_program': 'นานาชาติและภาษาอังกฤษ', 'reference': 'https://admission.ku.ac.th/majors/project/3/', 'created_at': '2025-01-25T16:37:01.806305', 'contents': 'N/A'}
Found ID: ace11b65-fe52-55b5-9218-b035bd9b08ed, Score: 0.3309437, Metadata: {'major': 'วศ.บ. สาขาวิชาวิศวกรรมโยธา (หลักสูตรภาษาอังกฤษ)', 'admission_round': '1/1', 'admission_program': 'นานาชาติและภาษาอังกฤษ', 'reference': 'https://admission.ku.ac.th/majors/project/3/', 'created_at': '2025-01-25T16:37:01.797005', 'conte